In [1]:
import numpy as np
import pandas as pd
import os
import json
import librosa
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
!apt install -y ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [3]:
SOURCE_PATH = "drive/MyDrive/GTZAN/genres_original"

JSON_PATH = 'drive/MyDrive/data.json'

sample_rate = 22050

TOTAL_SAMPLES = 29 * sample_rate

NUM_SLICES = 10
SAMPLES_PER_SLICE = int(TOTAL_SAMPLES / NUM_SLICES)

In [4]:
def preprocess_data(source_path, json_path):

    mydict = {
        "labels": [],
        "mfcc": []
        }

    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(source_path)):
        for file in filenames:
            # exclude corrupted wav file
            if os.path.join(dirpath, file) != 'drive/MyDrive/music-classification-data/genres_original/jazz/jazz.00054.wav':
                song, sr = librosa.load(os.path.join(dirpath, file), duration=29)
                for s in range(NUM_SLICES):
                    start_sample = SAMPLES_PER_SLICE * s
                    end_sample = start_sample + SAMPLES_PER_SLICE
                    mfcc = librosa.feature.mfcc(y=song[start_sample:end_sample], sr=sr, n_mfcc=13)
                    mfcc = mfcc.T
                    mydict["labels"].append(i-1)
                    mydict["mfcc"].append(mfcc.tolist())
   
    with open(json_path, 'w') as f:
        json.dump(mydict, f)
    f.close()

In [5]:
def load_data(json_path):

    with open(json_path, 'r') as f:
        data = json.load(f)
    f.close()

    X = np.array(data["mfcc"])
    y = np.array(data["labels"])

    return X, y

In [6]:
def prepare_datasets(inputs, targets, split_size):
    
    # Splitting sets
    inputs_train, inputs_val, targets_train, targets_val = train_test_split(inputs, targets, test_size=split_size)
    inputs_train, inputs_test, targets_train, targets_test = train_test_split(inputs_train, targets_train, test_size=split_size)
    
    # 3D input for CNN
    inputs_train = inputs_train[..., np.newaxis]
    inputs_val = inputs_val[..., np.newaxis]
    inputs_test = inputs_test[..., np.newaxis]
    
    return inputs_train, inputs_val, inputs_test, targets_train, targets_val, targets_test

In [7]:
def design_model(input_shape):

    model = tf.keras.models.Sequential([
        
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D((3,3), strides=(2,2), padding='same'),
        tf.keras.layers.BatchNormalization(),
        
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D((3,3), strides=(2,2), padding='same'),
        tf.keras.layers.BatchNormalization(),
        
        tf.keras.layers.Conv2D(32, (2,2), activation='relu'),
        tf.keras.layers.MaxPooling2D((3,3), strides=(2,2), padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'), 
        tf.keras.layers.Dense(len(np.unique(targets)), activation='softmax')
    ])

    return model

In [8]:
if __name__ == "__main__":

    # preprocess_data(source_path=SOURCE_PATH, json_path=JSON_PATH)
    
    inputs, targets = load_data(json_path=JSON_PATH)
    
    Xtrain, Xval, Xtest, ytrain, yval, ytest = prepare_datasets(inputs, targets, 0.2)

    input_shape = (Xtrain.shape[1], Xtrain.shape[2], 1)
    model = design_model(input_shape)

    model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.001),
                     loss='sparse_categorical_crossentropy',
                     metrics = ['acc']
                     )

    model.summary()

    history = model.fit(Xtrain, ytrain,
                        validation_data=(Xval, yval),
                        epochs=25,
                        batch_size=32
                        )

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 123, 11, 32)       320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 62, 6, 32)        0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 62, 6, 32)        128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 4, 32)         9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 2, 32)        0         
 2D)                                                             
                                                        

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/25
200/200 [==============================] - 15s 70ms/step - loss: 1.5725 - acc: 0.4510 - val_loss: 1.4319 - val_acc: 0.4850
Epoch 2/25
200/200 [==============================] - 18s 88ms/step - loss: 1.1462 - acc: 0.5906 - val_loss: 1.0645 - val_acc: 0.6086
Epoch 3/25
200/200 [==============================] - 13s 66ms/step - loss: 0.9752 - acc: 0.6574 - val_loss: 0.9472 - val_acc: 0.6617
Epoch 4/25
200/200 [==============================] - 13s 67ms/step - loss: 0.8584 - acc: 0.6931 - val_loss: 0.9176 - val_acc: 0.6737
Epoch 5/25
200/200 [==============================] - 13s 66ms/step - loss: 0.7724 - acc: 0.7249 - val_loss: 0.9902 - val_acc: 0.6707
Epoch 6/25
200/200 [==============================] - 14s 70ms/step - loss: 0.7103 - acc: 0.7463 - val_loss: 1.1052 - val_acc: 0.6376
Epoch 7/25
200/200 [==============================] - 14s 68ms/step - loss: 0.6495 - acc: 0.7719 - val_loss: 0.9296 - val_acc: 0.6827
Epoch 8/25
200/200 [==============================] - 14s 68ms

In [9]:
test_loss,test_acc=model.evaluate(Xtest,ytest,batch_size=128)
print("The test loss is ",test_loss)
print("The best accuracy is: ",test_acc*100)

13/13 [==============================] - 1s 48ms/step - loss: 0.7053 - acc: 0.8024
The test loss is  0.7053409814834595
The best accuracy is:  80.23765087127686


In [13]:
model.save('drive/MyDrive')

INFO:tensorflow:Assets written to: drive/MyDrive/assets
